# Code that works


In [13]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time 
import csv

import os
from dotenv import load_dotenv

dotenv_path = "C:/Users/alaha/Documents/GitHub/.env" 

# Load the .env file
load_dotenv(dotenv_path=dotenv_path)

True

In [14]:
chromedriver_path = "C:/Users/alaha/chromedriver.exe"

# Setup Chrome service and options
service = Service(chromedriver_path)
options = Options()

# Initialize the WebDriver
driver = webdriver.Chrome(service=service, options=options)

In [15]:
driver.get("https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin")

In [19]:
time.sleep(2)
email = driver.find_element(By.XPATH, "//input[@name='session_key']")
password = driver.find_element(By.XPATH, "//input[@name='session_password']")

email.send_keys(LINKEDIN_MAIL)
password.send_keys(LINKEDIN_PWD)

login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
login_button.click()
time.sleep(5)

In [35]:
# driver.get("https://www.linkedin.com/feed/?nis=true")

In [5]:
driver.get("https://www.linkedin.com/search/results/people/?currentCompany=%5B%221292%22%5D&keywords=data%20engineer&network=%5B%22S%22%2C%22O%22%5D&origin=FACETED_SEARCH&searchId=646dc372-74ad-44ec-be12-bdaf8eb44dbd&sid=YJR")

In [6]:
def process_page(driver):
    names = []
    try:
        connect_buttons = WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.XPATH, "//button[contains(@aria-label, 'Invite')]"))
        )

        for button in connect_buttons[:5]:  # Limit to first 5 buttons
            aria_label = button.get_attribute("aria-label")  # Get aria-label attribute
            if aria_label:
                # Extract the name, assuming the format "Invite [Name] to connect"
                name = aria_label.replace("Invite ", "").replace(" to connect", "").strip()
                names.append(name)

        print(f"Extracted Names: {names}")
    except Exception as e:
        print(f"Error while extracting names: {e}")
        go_to_next_page(driver)


    # Iterate through the first 5 names and click Connect only if verified
    for button, name in zip(connect_buttons, names):
        try:
            # Adjusted XPath to find a parent element related to badges
            parent_element = button.find_element(By.XPATH, "./ancestor::div[contains(@class, 'oHAGVpECrqpKKRajRuuzdOGMowaLXTyMuJCuQ')]")
            try:   
                print(parent_element)
                verified_badge = parent_element.find_element(By.XPATH, ".//button[contains(@class, 'reusable-search-modal-badge__verified-badge-button')]")
                is_verified = True
                print(f"{name} is verified.")
            except Exception as e:
                is_verified=False
                print(f"Could not add a note for {name}: {e}")

            # Only proceed if the user is verified
            if is_verified:
                print(f"Sending connection request to: {name}")
                button.click()
                time.sleep(2)  # Allow modal to load

                # Check for "Add a Note" button
                try:
                    add_note_button = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'Add a note')]"))
                    )
                    add_note_button.click()
                    print(f"Clicked Add a Note for {name}")

                    # Add a note and send request
                    note_text_area = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, "//textarea[@id='custom-message']"))
                    )
                    custom_message = (
                        f"Hi {name}, I came across your profile and I am really inspired by it. I would love to connect for a resume review & discuss day-to-day activities at your organization with a possibility to potential opportunities. Can we please connect?"
                    )
                    note_text_area.send_keys(custom_message)
                    print(f"Typed a custom note for {name}")

                    send_button = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'Send invitation')]"))
                    )
                    send_button.click()
                    print(f"Connection request sent to {name}")

                    file_path = "names.txt"  # Define the file path
                    with open(file_path, "a") as file:
                        file.write(f"{name}\n")  # Write the name followed by a newline
                        print(f"{name} added to {file_path}.")
                    
                except Exception as e:
                    print(f"Could not add a note for {name}: {e}")
            
        except Exception as e:
            print(f"Error while processing {name}: {e}")

In [7]:
def go_to_next_page(driver):
    try:
        next_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'Next') and contains(@class, 'artdeco-pagination__button--next')]"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        time.sleep(1)  # Brief pause to ensure button is in view
        next_button.click()
        time.sleep(3)  # Wait for new page to load
        return True
    except Exception as e:
        print(f"Could not go to next page: {e}")
        return False

In [8]:
def main(driver, max_pages=30):
    page_number = 1
    
    while page_number <= max_pages:
        print(f"\nProcessing page {page_number}")
        
        # Process current page
        # if not process_page(driver):
        #     print("Error processing page. Stopping.")
        #     break
        process_page(driver)
        # Try to go to next page
        if not go_to_next_page(driver):
            print("Reached last page or error occurred. Stopping.")
            break
        
        page_number += 1
        print(f"Moving to page {page_number}")

        time.sleep(5)

In [11]:
try:
    main(driver)
except Exception as e:
    print(f"Script stopped due to error: {e}")


Processing page 1
Extracted Names: ['Emma H.', 'Nithin Kumar E']
<selenium.webdriver.remote.webelement.WebElement (session="54311978f9fe6fb1e0bb0ec36ad78483", element="f.D544F75282E30C1045FFDEA8BA8A14F3.d.3035AF6D88820A880D7C47B3A3C478F4.e.67873")>
Could not add a note for Emma H.: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//button[contains(@class, 'reusable-search-modal-badge__verified-badge-button')]"}
  (Session info: chrome=132.0.6834.111); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF61C6EFB05+28789]
	(No symbol) [0x00007FF61C6586E0]
	(No symbol) [0x00007FF61C4F592A]
	(No symbol) [0x00007FF61C54930E]
	(No symbol) [0x00007FF61C5495FC]
	(No symbol) [0x00007FF61C53C6EC]
	(No symbol) [0x00007FF61C56F47F]
	(No symbol) [0x00007FF61C53C5B6]
	(No symbol) [0x00007FF61C56F650]
	(No symbol) [0x00007FF61C58F654]


# Jobs


In [6]:
# Initialize WebDriver (continuing from your previous setup)
time.sleep(5)
driver.get("https://www.linkedin.com/jobs")


In [7]:
search_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "jobs-search-box__text-input"))
)

# Remove the disabled attribute using JavaScript
driver.execute_script("arguments[0].removeAttribute('disabled');", search_box)

# Now interact with the search box
search_box.send_keys("Data Engineer")
search_box.send_keys(Keys.RETURN)
time.sleep(5)

In [11]:
csv_file = "non_easy_apply_jobs.csv"
with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Job Title", "Company Name", "Location", "Job Link"])

In [ ]:

try:
    # Wait for the job cards to load
    job_cards = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".job-card-container__link"))
    )
    
    print(f"Found {len(job_cards)} job cards.")

    for index, job_card in enumerate(job_cards[1:3], start=1):
        # Extract the href link from the job card
        job_link = job_card.get_attribute("href")
        
        print(f"\nOpening job card {index}: {job_link}")

        # Open the job link in a new tab
        driver.execute_script("window.open(arguments[0], '_blank');", job_link)
        
        # Switch to the new tab
        driver.switch_to.window(driver.window_handles[-1])
        
        # Perform actions on the job detail page
        try:
            # Wait for the job description or other elements to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "jobs-description-content"))
            )
            print(f"Job {index} details page loaded.")
        except Exception as e:
            print(f"Failed to load details for job {index}: {e}")
        
        # Close the tab and switch back to the original tab
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()

Found 25 job cards.

Opening job card 1: https://www.linkedin.com/jobs/view/4103772587/?eBP=CwEAAAGULL5r0jVVi89mi4HnDtwTOPdoQr-nJiqFti73VEejZOm6hwjeqVyd69QJNPotS49fGzeZ2giAVOrV_06KVywMd6Ovdcp6OieVp5lLaT6CAlYP3yN4WsEZIVnhO4pVUFwqzwXnNEDOBEoevsBFXgd-D6vQ3AS2Bte5RVrKWbmNNZdYu8W5NrGqd7j8k-7nFKKIbnOyDcFOkufz-BGhA2El4r6sDacTOS6sbjzTsYowNZ5kpcHlkkPZlj7t8Y7Kp5Qox5cKq0Lz4M7KCxR_bE02yEv0i9iXt4V3eHO6BFcdiOXXR5v9RmNQhvyj9Zp981zX9R3s6N8HSj_zOFzlRWK8nGlZEq2k2ven9MwoYAkW1OibKUXHN8UevJ2wUS3NhmWbW6o4QLhdx4hmpShX9CCk2r_j_e8ogwNIwoIrVGy7ORY5U_o-HIrq3WGuIiAg2_aKZDdJzzRfp4nwWko_gLw5pP_mPRZy1VqM-nz2vSB3_3VKkIg9i1sOotskInUGikPYLMw&refId=Ceg8At4s0pz90Tluq6%2BI6A%3D%3D&trackingId=yWIs2CdR53G3E3edyF5g2A%3D%3D&trk=flagship3_search_srp_jobs
Failed to load details for job 1: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7FE1BFB05+28789]
	(No symbol) [0x00007FF7FE1286E0]
	(No symbol) [0x00007FF7FDFC592A]
	(No symbol) [0x00007FF7FE01930E]
	(No symbol) [0x00007FF7FE0195FC]
	(No symbol) [0x00007FF7FE0628A7]
	(No 

# By linkedIn API

In [12]:
pip install linkedin-api


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\alaha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
from linkedin_api import Linkedin
import os
from time import sleep

In [14]:
api = Linkedin(LINKEDIN_MAIL, LINKEDIN_PWD)

In [32]:
from linkedin_api import Linkedin
import datetime
from time import sleep
import pandas as pd
import os

def search_recent_jobs(job_title, location, time_posted=24):
    """
    Search for job postings on LinkedIn with specific criteria
    
    Args:
        email (str): LinkedIn account email
        password (str): LinkedIn account password
        job_title (str): Job title to search for
        location (str): Location to search in
        time_posted (int): Hours since job was posted
        
    Returns:
        list: List of job postings matching criteria
    """
    try:
        # Initialize the API client
        api = Linkedin(LINKEDIN_MAIL, LINKEDIN_PWD)
        
        # Calculate the timestamp for 24 hours ago
        time_threshold = datetime.datetime.now() - datetime.timedelta(hours=time_posted)
        
        # Parameters for job search
        params = {
            "keywords": job_title,
            "location": location,
            "locationId": "us:0",  # United States
            "timePosted": "past-24h",
            "f_TPR": "r86400",  # Posted in last 24 hours
            "distance": "50",
            "f_WT": "2",  # Full-time jobs
            "position": 0,
            "pageNum": 0
        }
        
        all_jobs = []
        max_pages = 1  # Limit the number of pages to avoid rate limiting
        
        for page in range(max_pages):
            params["pageNum"] = page
            
            # Add delay between requests to avoid rate limiting
            if page > 0:
                sleep(2)
            
            # Search for jobs
            jobs = api.search_jobs(**params)
            
            if not jobs:
                break
                
            for job in jobs:
                print(job)
                # Get detailed job information
                job_detail = api.get_job(job['entityUrn'].split(':')[-1])
                
                # Extract relevant information
                job_info = {
                    'title': job_detail.get('title', ''),
                    'company': job_detail.get('companyDetails', {}).get('company', {}).get('name', ''),
                    'location': job_detail.get('formattedLocation', ''),
                    'listing_url': f"https://www.linkedin.com/jobs/view/{job_detail.get('entityUrn', '').split(':')[-1]}",
                    'posted_time': job_detail.get('listedAt', ''),
                    'description': job_detail.get('description', {}).get('text', ''),
                    'applicant_count': job_detail.get('applicantCount', 0),
                    'seniority_level': job_detail.get('seniority', {}).get('name', ''),
                    'employment_type': job_detail.get('employmentStatus', ''),
                    'experience_level': job_detail.get('experience', ''),
                }
                
                # Convert timestamp to datetime
                posted_time = datetime.datetime.fromtimestamp(job_info['posted_time'] / 1000)
                
                # Only include jobs posted within the specified timeframe
                if posted_time > time_threshold:
                    all_jobs.append(job_info)
            
            print(f"Processed page {page + 1}, found {len(all_jobs)} jobs so far")
            
        return all_jobs
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return []

def save_to_csv(jobs, filename="data_engineer_jobs.csv"):
    """
    Save job listings to CSV file
    """
    df = pd.DataFrame(jobs)
    df.to_csv(filename, index=False)
    print(f"Jobs saved to {filename}")

def main():
    # Search parameters
    JOB_TITLE = "Data Engineer"
    LOCATION = "United States"
    HOURS = 24
    
    # Search for jobs
    jobs = search_recent_jobs(
        job_title=JOB_TITLE,
        location=LOCATION,
        time_posted=HOURS
    )
    
    if jobs:
        print(f"\nFound {len(jobs)} job postings")
        # Save results to CSV
        save_to_csv(jobs)
        
        # Display some statistics
        companies = len(set([job['company'] for job in jobs]))
        locations = len(set([job['location'] for job in jobs]))
        print(f"\nSummary:")
        print(f"Total companies: {companies}")
        print(f"Total locations: {locations}")
    else:
        print("No jobs found matching the criteria")

if __name__ == "__main__":
    main()

{'trackingUrn': 'urn:li:jobPosting:4124142306', 'repostedJob': True, 'title': 'Big Data Developer', '$recipeTypes': ['com.linkedin.deco.recipe.anonymous.Anon1578943416'], 'posterId': '414283750', '$type': 'com.linkedin.voyager.dash.jobs.JobPosting', 'contentSource': 'JOBS_PREMIUM_OFFLINE', 'entityUrn': 'urn:li:fsd_jobPosting:4124142306'}
{'trackingUrn': 'urn:li:jobPosting:4127537705', 'repostedJob': False, 'title': 'Business Intelligence Developer', '$recipeTypes': ['com.linkedin.deco.recipe.anonymous.Anon1578943416'], 'posterId': '1007205883', '$type': 'com.linkedin.voyager.dash.jobs.JobPosting', 'contentSource': 'JOBS_PREMIUM_OFFLINE', 'entityUrn': 'urn:li:fsd_jobPosting:4127537705'}
{'trackingUrn': 'urn:li:jobPosting:4125855481', 'repostedJob': False, 'title': 'Data Integration Engineer', '$recipeTypes': ['com.linkedin.deco.recipe.anonymous.Anon1578943416'], 'posterId': '47493562', '$type': 'com.linkedin.voyager.dash.jobs.JobPosting', 'contentSource': 'JOBS_PREMIUM_OFFLINE', 'entity

KeyboardInterrupt: 